In [59]:
import pandas as pd
import numpy as np
import progressbar
from scipy.special import digamma, gammaln
from math import log, exp
from numpy.linalg import norm
from sklearn.cluster import KMeans


class Raw_SBM(object):
    def __init__(self, X):
        self.X = X
        self.N = len(X)
        
        
        
class HofWig_SBM(Raw_SBM):
    def __init__(self, n, Q, alpha = None, p_epsilon = 0.1, p_lambda = 0.9):
        
        if alpha is None:
            a = np.random.uniform(size=Q)
            alpha = a/a.sum()
        if len(alpha) != Q: raise ValueError("Alpha has not the same length as Q")

        # Defines class
        Z = np.zeros(n, dtype=np.uint)
        i = 0
        for q, n_q in enumerate(np.random.multinomial(n, alpha)):
            for j in range(n_q):
                Z[i] = q
                i += 1

        # Probability matrix of connection between classes
        pi = np.zeros((Q,Q)) + p_epsilon + np.diag([p_lambda - p_epsilon]*Q)

        # Matrix of connections
        X = np.zeros((n,n), dtype = bool)
        for i in range(n):
            q_i = Z[i]
            for j in range(i):
                q_j = Z[j]
                bound = bool(np.random.binomial(1, pi[q_i][q_j]))
                X[i][j] = bound
                X[j][i] = bound

        self.Z = Z
        self.Q = Q
        
        super(HofWig_SBM, self).__init__(X)
        
        
        

class SVBM(object):
    
    def __init__(self, net, Q = None):
        self.net = net
        self.Q = net.Q if Q is None else Q
        
    def _init_tau(self):
        #Shortcut
        net = self.net 
        Q, N = self.Q, self.net.N
        
        kmeans = KMeans(n_clusters=Q).fit(net.X)
        self.tau = np.zeros((N, Q))
        for i, q in enumerate(kmeans.labels_):
            self.tau[i][q] = 1
            
        return self.tau
            
    def _init_theta(self):
        #Shortcut
        X = self.net.X
        N = self.net.N
        Q = self.Q
        
        self.n = np.zeros(Q)
        self.eta = np.zeros((Q, N))
        self.zeta = np.zeros((Q, N))
        for q in range(Q):
            # N
            s_n = 0.5
            for i in range(N):
                s_n += self.tau[i][q]
            self.n[q] = s_n

            # Eta
            for l in range(Q):
                s_eta, s_zeta = 0.5, 0.5 # Default value
                for j in range(N):
                    for i in range(N):
                        if (q == l and j <= i): break
                        if i == j: continue

                        s_eta += X[i][j]*self.tau[i][q]*self.tau[j][l]
                        s_zeta += (1-X[i][j])*self.tau[i][q]*self.tau[j][l]

                self.eta[q][l] = s_eta
                self.zeta[q][l] = s_zeta

                if (s_eta < 0):
                    raise ValueError(s_eta)           
                if (s_zeta < 0):
                    raise ValueError(s_zeta)    
        
        return self.n, self.eta, self.zeta
    
    
    def run(self):
        #Shortcut
        Q, N, X = self.Q, self.net.N, self.net.X
         
        # Init
        print("Initialisation...")
        self._init_tau()
        n, eta, zeta = self._init_theta()
        
        print("Running...")
        for iterr in range(31):
            delta = 0

            # Maximisation
            for i in range(N):
                tau_i = np.zeros(Q)
                for q in range(Q):
                    p = exp(digamma(n[q])-digamma(sum(n)))
                    for j in range(N):
                        if (i==j): continue
                        for l in range(Q):
                            a = digamma(zeta[q][l])\
                                 - digamma(eta[q][l] + zeta[q][l])\
                                 + X[i][j]*(digamma(eta[q][l]) - digamma(zeta[q][l]))
                            a *= self.tau[j][l]
                            p *= exp(a)
                    tau_i[q] = p
                tau_i *= 1/sum(tau_i)
                delta += norm(tau_i - self.tau[i])
                self.tau[i] = tau_i

            # Update intern variable
            self.lower_bound = self._lower_bound_aprox()
            
            # Show
            print("Iteration n°" + str(iterr+1) + " :"\
                  + "\n\tLower bound : " + str(self.lower_bound)\
                  + "\n\tDelta : " + str(delta))
            
            # Break condition
            if (30 <= iterr):
                print("No convergence !")
                break
            if (delta < 10E-4):
                print("EM has converged")
                break
                
        return self
                
    def compare(self):
        Z = s.net.Z
        if Z is None:
            raise ValueError("This network does not have a Z variable. SVBM can't compare !")
            
        res = []
        for i, z in enumerate(Z):
            res.append([z, self.tau[i].argmax()])
        return pd.DataFrame(res, columns=["Z", "Tau"]).groupby(["Z", "Tau"]).size()

    
    def _lower_bound_aprox(self):
        # Shortcut
        N, eta, zeta, tau = self.n, self.eta, self.zeta, self.tau
        n, Q = tau.shape
        
        s = 0
        
        s += gammaln(Q*0.5) - gammaln(sum(N)) - Q*gammaln(0.5)
        for q in range(Q):
            s += gammaln(N[q])

        s += (Q*(Q+1)*0.5)*(gammaln(1) - 2*gammaln(0.5))
        for l in range(Q):
            for q in range(l+1):
                s += gammaln(eta[q][l]) + gammaln(zeta[q][l]) - gammaln(eta[q][l]*zeta[q][l])

        for i in range(n):
            for q in range(Q):
                s -= tau[i][q]*log(tau[i][q])

        return s

In [62]:
net = HofWig_SBM(100, 2, p_epsilon = 0.3, p_lambda = 0.7)
s = SVBM(net)
s.run().compare()

Initialisation...
Running...
Iteration n°1 :
	Lower bound : -26539806.8625
	Delta : 5.4705162366e-08
EM has converged


Z  Tau
0  0      34
1  1      66
dtype: int64

# Build real network

In [ ]:
def load(name):
    return pd.read_pickle("data/" + name + ".pickle")

In [ ]:
emails = load("emails")
emails.head()

In [ ]:
ms = load("messages")
ms.head()

In [ ]:
rs = load("recipients")
rs.head()

In [ ]:
rs = rs.loc[rs.m_id.isin(ms.index.values)]
X_bounds = rs.join(ms, on = "m_id", rsuffix="_sender")[["e_id_sender", "e_id"]].drop_duplicates().rename(columns = {"e_id_sender" : "sender_id", "e_id" : "recipient_id"})

In [ ]:
X_bounds.head()

In [ ]:
n = max(ms.e_id.max(), rs.e_id.max()) + 1
X = np.zeros((n,n), dtype=bool)
X.shape

bar = progressbar.ProgressBar(max_value=X_bounds.shape[0])
for index, row in bar(X_bounds.iterrows()):
    a,b = row["sender_id"], row["recipient_id"]
    X[a][b] = True
    X[b][a] = True

In [ ]:
from numpy.linalg import norm
norm(tau_old - tau_new)